In [6]:
import requests
import time
import random
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import json

In [7]:
headers = {
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "Accept-Encoding": "gzip, deflate, br, zstd",
    "Accept-Language": "es-ES,es;q=0.9",
    "Cache-Control": "max-age=0",
    "cookie": 're_uuid=86efd51c-a8a2-40b1-81b2-6af2bd2cde4d; euconsent-v2=CQIv8oAQIv8oAAHABBENBRFsAP_gAEPgAAiQKoNX_G__bWlr8X73aftkeY1P9_h77sQxBhfJE-4FzLvW_JwXx2ExNA36tqIKmRIAu3TBIQNlGJDURVCgaogVryDMaEyUoTNKJ6BkiFMRM2dYCFxvm4tj-QCY5vr991dx2B-t7dr83dzyy4xHn3a5_2S0WJCdA5-tDfv9bROb-9IOd_x8v4v4_F_pE2_eT1l_tWvp7D9-cts__XW89_fff_9Pn_-uB_-_3_vf_BU4Akw0KiAMsCQkINAwggQAqCsICKBAEAACQNEBACYMCnYGAC6wkQAgBQADBACAAEGQAIAAAIAEIgAgAKBAABAIFAAGABAMBAAQMAAIALAQCAAEB0DFMCCAQLABIzIiFMCEIBIICWyoQSAIEFcIQizwCIBETBQAAAAAFIAAgLBYHEkgJUJBAFxBtAAAQAIBBAAUIJOTAAEAZstQeDBtGVpgGD5gkQ0wDIAiCMgAAAAA.f_wACHwAAAAA; didomi_token=eyJ1c2VyX2lkIjoiMTkzNmUyMTYtZGYzMC02MzJkLThjMTMtMGM4NzhmNzNmMmFiIiwiY3JlYXRlZCI6IjIwMjQtMTEtMjdUMTQ6NTg6MjQuODgzWiIsInVwZGF0ZWQiOiIyMDI0LTExLTI3VDE0OjU4OjI4Ljc0NVoiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiZ29vZ2xlIiwiYzpnb29nbGVhbmEtNFRYbkppZ1IiLCJjOmFkZXZpbnRhLW1vdG9yLW1vdG9yIiwiYzptZXRhcGxhdGYtTlJlVnBtTFEiLCJjOmNyaXRlby1QNDhlR1QydyIsImM6Z29vZ2xlaXJlLWZmS2FQYVJEIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbImRldmljZV9jaGFyYWN0ZXJpc3RpY3MiLCJnZW9sb2NhdGlvbl9kYXRhIiwidHJhbnNmZXItdG8tbW90b3IiLCJ1c29kZWxvcy1RcExwTThqVyJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6Mn0=; ab.storage.deviceId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3A43729e38-14f9-6dbd-da2a-ec3d01719b31%7Ce%3Aundefined%7Cc%3A1732719505639%7Cl%3A1733239374133; ab.storage.sessionId.c9f9127d-989c-45fe-96f2-f36434db5025=g%3Af153a8e0-a024-9099-012a-991a1543e16a%7Ce%3A1733241174863%7Cc%3A1733239374132%7Cl%3A1733239374863; segment_ga_WLSRR9KS1C=GS1.1.1733239373.1.0.1733239373.60.0.0; segment_ga=GA1.1.401735641.1733239374; ajs_anonymous_id=b5c7048f-58b8-46ab-bc83-773ac359cc73; borosTcf=eyJwb2xpY3lWZXJzaW9uIjoyLCJjbXBWZXJzaW9uIjoxLCJwdXJwb3NlIjp7ImNvbnNlbnRzIjp7IjEiOnRydWUsIjIiOnRydWUsIjMiOnRydWUsIjQiOnRydWUsIjUiOnRydWUsIjYiOnRydWUsIjciOnRydWUsIjgiOnRydWUsIjkiOnRydWUsIjEwIjp0cnVlfX0sInNwZWNpYWxGZWF0dXJlcyI6eyIxIjp0cnVlfX0=; _gcl_au=1.1.795359020.1733239374; AMCV_05FF6243578784B37F000101%40AdobeOrg=-408604571%7CMCIDTS%7C20061%7CMCMID%7C13264459680476315102412969132628763965%7CMCAAMLH-1733844174%7C6%7CMCAAMB-1733844174%7C6G1ynYcLPuiQxYZrsz_pkqfLG9yMXBpb2zX5dvJdYQJzPXImdj0y%7CMCOPTOUT-1733246575s%7CNONE%7CMCAID%7CNONE%7CvVersion%7C4.6.0; kanirante-adv-aauid=076a3cd7-af3c-4a81-921c-d8a7b01b3d44; AMCVS_05FF6243578784B37F000101%40AdobeOrg=1; __rtbh.uid=%7B%22eventType%22%3A%22uid%22%2C%22id%22%3A%22unknown%22%2C%22expiryDate%22%3A%222025-12-03T15%3A22%3A54.984Z%22%7D; __rtbh.lid=%7B%22eventType%22%3A%22lid%22%2C%22id%22%3A%22KhrsxWw2uOmhnZAYTuzU%22%2C%22expiryDate%22%3A%222025-12-03T15%3A22%3A54.984Z%22%7D; sui_1pc=173323937504654011CCE3D4283A64469FCAB3CE77E57510276474FA; adit-xandr-id=0; _fbp=fb.1.1733239375189.920492403578503430; cto_bundle=nykbx191eERLSnNPRTJmMUR0d3phOG1WTDVXTzNucUtIJTJGSnkzZjdhJTJGZmpzdnZtUFI2cUlCM0dHZVZCT0VlVlU5a3hkcEx1b2FjRGUlMkJQbkQ3c2xPbEFJY2FKUHlCSTRMYmdNZXI1MWNiYVQyRnBSM0wzYURIWFBBR0FUbHZRWGdqMER1d2tSRmgxUkNySnEzRG8lMkY2S2c5ZURWUSUzRCUzRA; __gads=ID=ee5b6d4d292f80d8:T=1733239375:RT=1733239375:S=ALNI_MbVRd5AiMzPeuDHhaUY8M-7cde--w; __gpi=UID=00000faf6bc065db:T=1733239375:RT=1733239375:S=ALNI_MYecin-_Fwsa7V_U8rO1qNE4nTbOQ; __eoi=ID=1bae3587855210a1:T=1733239375:RT=1733239375:S=AA-Afja8E7nXgwwbrpqbzkBjpDMN',
    "Referer": "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/l",
    "Sec-Ch-Device-Memory": "4",
    "Sec-Ch-Ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "Sec-Ch-Ua-Full-Version-List": '"Not)A;Brand";v="99.0.0.0", "Google Chrome";v="127.0.6533.120", "Chromium";v="127.0.6533.120"',
    "Sec-Ch-Ua-Mobile": "?0",
    "Sec-Fetch-Dest": "document",
    "Sec-Fetch-Mode": "navigate",
    "Sec-Fetch-Site": "same-origin",
    "Sec-Fetch-User": "?1",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
    "X-Height": "9999",  # Establece el valor de height
    "X-Width": "1920",   # Puedes agregar otros parámetros, como width
    "X-Depth": "500",    # Ejemplo de parámetro depth, si es necesario
    "X-Color": "blue",   # Ejemplo de parámetro de color, si lo deseas
    "X-Orientation": "landscape"  # Otro ejemplo, si necesitas un parámetro de orientación
}


In [ ]:
# Crear una sesión
def check_error_page(html_page):
    # Patrón para buscar frases comunes en una página de error
    patrones_error = [
        r"sentimos la interrupciã\x93n",  # Detecta el mensaje de "Sentimos la interrupción"
    ]
    
    # Convertir el HTML a minúsculas para hacerlo insensible a mayúsculas
    html_lower = html_page.lower()
    # Buscar cada uno de los patrones de error en el HTML
    for patron in patrones_error:
        if re.search(patron, html_lower):
            return True
    return False
    
# Función para obtener los detalles del inmueble
def extract_property_info(property_data):
    # Titulo del inmueble: Se puede obtener desde la dirección del detalle
    titulo = f"{property_data['address']['city']} - {property_data['address']['district']}"

    # Precio (lo extraemos del campo 'price')
    precio = property_data['price']

    # Habitaciones: 'rooms' en las características
    habitaciones = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'rooms'), None)

    # Convertir a int si no es None
    habitaciones = int(habitaciones) if habitaciones is not None else 0

    # Baños: 'bathrooms' en las características
    banos = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'bathrooms'), None)

    # Superficie: 'surface' en las características
    superficie = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'surface'), None)

    # Convertir a int si no es None
    superficie = int(superficie) if superficie is not None else 0

    # Planta: 'floor' en las características
    planta = next((feature['value'] for feature in property_data['features'] if feature['key'] == 'floor'), None)

    # Convertir a int si no es None
    planta = int(planta) if planta is not None else 0

    # Enlace: detalle completo
    full_link = property_data['detail'].get('es-ES', '')

    return {
        "titulo": titulo,
        "precio": precio,
        "habitaciones": habitaciones,
        "banos": banos,
        "superficie": superficie,
        "planta": planta,
        "enlace": full_link,
        'fuente': 'fotocasa',
    }

session = requests.Session()
session.headers.update(headers)

# URL base
base_url = "https://www.fotocasa.es/es/alquiler/viviendas/murcia-provincia/todas-las-zonas/"

# Función para verificar si es una página de error
def check_error_page(html_page):
    patrones_error = [r"sentimos la interrupciã\x93n"]
    html_lower = html_page.lower()
    for patron in patrones_error:
        if re.search(patron, html_lower):
            return True
    return False

# Función para extraer los datos de un inmueble según JSON
def extract_property_info(property_data):
    try:
        titulo = f"{property_data['address']['city']} - {property_data['address']['district']}"
        precio = property_data.get('price', None)
        habitaciones = int(next((f['value'] for f in property_data['features'] if f['key'] == 'rooms'), 0))
        banos = int(next((f['value'] for f in property_data['features'] if f['key'] == 'bathrooms'), 0))
        superficie = int(next((f['value'] for f in property_data['features'] if f['key'] == 'surface'), 0))
        planta = int(next((f['value'] for f in property_data['features'] if f['key'] == 'floor'), 0))
        full_link = property_data['detail'].get('es-ES', '')
        return {
            "titulo": titulo,
            "precio": precio,
            "habitaciones": habitaciones,
            "banos": banos,
            "superficie": superficie,
            "planta": planta,
            "enlace": full_link,
            "fuente": "fotocasa"
        }
    except Exception as e:
        print(f"Error extrayendo datos del inmueble: {e}")
        return None

# Función para manejar diferentes selectores según la estructura de la página
def extract_data_with_selectors(html, json_data=None):
    inmuebles = []

    # Intentar extraer desde JSON
    if json_data:
        try:
            real_estates = json_data['initialSearch']['result']['realEstates']
            for inmueble in real_estates:
                info = extract_property_info(inmueble)
                if info:
                    inmuebles.append(info)
        except KeyError:
            print("El JSON no contiene la clave esperada para 'realEstates'.")

    # Intentar extraer desde HTML usando BeautifulSoup (alternativa)
    try:
        soup = BeautifulSoup(html, 'html.parser')
        cards = soup.find_all('article', class_='re-CardPackPremium')
        for card in cards:
            titulo = card.find('h3', class_='re-CardHeader')
            precio = card.find('span', class_='re-CardPrice')
            enlace = card.find('a', class_='re-CardPackPremium-info-container')
            if titulo and precio and enlace:
                inmuebles.append({
                    "titulo": titulo.get_text(strip=True),
                    "precio": precio.get_text(strip=True),
                    "habitaciones": "N/A",
                    "banos": "N/A",
                    "superficie": "N/A",
                    "planta": "N/A",
                    "enlace": f"https://www.fotocasa.es{enlace['href']}",
                    "fuente": "fotocasa"
                })
    except Exception as e:
        print(f"Error extrayendo datos con BeautifulSoup: {e}")

    return inmuebles

# Paginación y scraping
page = 1
all_listings = []
page_limit = 30

while page < page_limit:
    try:
        print(f"Procesando página {page}...")
        response = session.get(f"{base_url}l/{page}")
        response.raise_for_status()
        error_page = check_error_page(response.text)
        if error_page:
            print("Bloqueado por Fotocasa, cambie la cookie")
            break

        # Intentar extraer JSON
        json_data = None
        match = re.search(
            r'window\.__INITIAL_PROPS__ = JSON\.parse\("((?:[^"\\]|\\.)*)"\);',
            response.text
        )
        if match:
            escaped_json = match.group(1)
            try:
                json_data = json.loads(escaped_json.encode('utf-8').decode('unicode_escape'))
            except json.JSONDecodeError:
                print("Error al decodificar el JSON")

        # Extraer datos usando múltiples estrategias
        inmuebles = extract_data_with_selectors(response.text, json_data)
        all_listings.extend(inmuebles)

        # Pasar a la siguiente página
        page += 1
        time.sleep(random.uniform(1, 5))  # Espera aleatoria para evitar bloqueos

    except Exception as e:
        print(f"Error procesando la página {page}: {e}")
        break

# Convertir a DataFrame
df = pd.DataFrame(all_listings)
df

Procesando página 1...
Procesando página 2...
Procesando página 3...
Procesando página 4...
Procesando página 5...
Procesando página 6...
Bloqueado por fotocasa, cambie la cookie


,titulo,precio,habitaciones,banos,superficie,planta,enlace,fuente
0,Murcia Capital - Norte,580 €,1,1,49,4,/es/alquiler/vivienda/murcia-capital/ascensor-...,fotocasa
1,La Manga del Mar Menor - Entremares,650 €,2,1,51,7,/es/alquiler/vivienda/la-manga-del-mar-menor/a...,fotocasa
2,Cartagena - Cartagena ciudad,800 €,2,1,87,9,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
3,Murcia Capital - Pedanías Este,695 €,2,1,75,6,/es/alquiler/vivienda/murcia-capital/aire-acon...,fotocasa
4,Murcia Capital - Centro,680 €,2,1,63,0,/es/alquiler/vivienda/murcia-capital/san-andre...,fotocasa
...,...,...,...,...,...,...,...,...
145,Murcia Capital - Pedanías Oeste,790 €,3,2,100,0,/es/alquiler/vivienda/murcia-capital/terraza-a...,fotocasa
146,Mazarrón - Puerto de Mazarrón,2.950 €,3,2,280,0,/es/alquiler/vivienda/mazarron/aire-acondicion...,fotocasa
147,Cartagena - Cartagena ciudad,1.200 €,3,2,120,0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
148,Cartagena - Cartagena ciudad,900 €,3,1,83,0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa


In [9]:
df

,titulo,precio,habitaciones,banos,superficie,planta,enlace,fuente
0,Murcia Capital - Norte,580 €,1,1,49,4,/es/alquiler/vivienda/murcia-capital/ascensor-...,fotocasa
1,La Manga del Mar Menor - Entremares,650 €,2,1,51,7,/es/alquiler/vivienda/la-manga-del-mar-menor/a...,fotocasa
2,Cartagena - Cartagena ciudad,800 €,2,1,87,9,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
3,Murcia Capital - Pedanías Este,695 €,2,1,75,6,/es/alquiler/vivienda/murcia-capital/aire-acon...,fotocasa
4,Murcia Capital - Centro,680 €,2,1,63,0,/es/alquiler/vivienda/murcia-capital/san-andre...,fotocasa
...,...,...,...,...,...,...,...,...
145,Murcia Capital - Pedanías Oeste,790 €,3,2,100,0,/es/alquiler/vivienda/murcia-capital/terraza-a...,fotocasa
146,Mazarrón - Puerto de Mazarrón,2.950 €,3,2,280,0,/es/alquiler/vivienda/mazarron/aire-acondicion...,fotocasa
147,Cartagena - Cartagena ciudad,1.200 €,3,2,120,0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa
148,Cartagena - Cartagena ciudad,900 €,3,1,83,0,/es/alquiler/vivienda/cartagena/aire-acondicio...,fotocasa


In [10]:
#df.to_csv('alquileres_fotocasa.csv')